In [1]:
!pip3 install torch torchvision pillow

In [2]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image


In [3]:
import sys
import os
sys.path.append('/Users/apoorvagayatrik/VJKLT2DPCA/ESRGAN')


In [4]:
import RRDBNet_arch as arch

In [5]:
model_path='/Users/apoorvagayatrik/VJKLT2DPCA/ESRGAN/models/RRDB_ESRGAN_x4.pth'

In [6]:
device=torch.device('cpu')

In [7]:
model=arch.RRDBNet(3,3,64,23,gc=32)

In [8]:
model.load_state_dict(torch.load(model_path,map_location=device),strict=True)

<All keys matched successfully>

In [9]:
model.eval()

RRDBNet(
  (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RRDB_trunk): Sequential(
    (0): RRDB(
      (RDB1): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (RDB2): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), str

In [10]:
model = model.to(device)


In [11]:
def apply_esrgan(frame):
    
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    
    rgb_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    pil_image=Image.fromarray(rgb_frame)
    
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])
    
    input_tensor=transform(pil_image).unsqueeze(0)
    
    input_tensor=input_tensor.to(device)
    
    with torch.no_grad():
        output_tensor=model(input_tensor).squeeze().cpu()
        
        
    output_image=output_tensor.clamp(0,1).numpy()
    output_image=np.transpose(output_image,(1,2,0))
    output_image=(output_image*255.0).astype(np.uint8)
    
    enchanced_frame=cv2.cvtColor(output_image,cv2,COLOR_RGB2BGR)
    enhanced_frame = cv2.resize(enhanced_frame, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_LINEAR)

    return enchanced_frame

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame =cap.read()
    
    if not ret:
        print("Error: Failed to capture frame from webcam.")
        break
        
    print("Frame captured successfully.")

        
    enhanced_frame=apply_esrgan(frame)
    
    cv2.imshow('Enhanced Frame',enhanced_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break00
        
        
cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release

Frame captured successfully.


In [3]:
if torch.cuda.is_available():
    print("CUDA is available! Running on GPU.")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"CUDNN version: {torch.backends.cudnn.version()}")
    print(f"CUDA devices: {torch.cuda.get_device_name(0)}")  # Get the name of the GPU
else:
    print("CUDA is not available. Running on CPU.")


CUDA is not available. Running on CPU.


In [2]:
import torch